# Telecom Churn Prediction Model

## Project Life Cycle
1. Understand Problem Statement
2. Data Collection
3. Data Checks and validation
4. Exploratory Data Analysis (EDA)
5. Data Pre-processing
6. Model Training & Testing
7. Model Evaluation - Choosing the best model

## 1. Understand the Problem Statement: 
-> *What is churn?* Churn occurs when a customer stops using the company's services and leaves for another provider.
This is bad for the company because acquiring new customers is often more expensive than keeping existing ones.


*Goal*:
This project involves identifying patterns and factors contributing to customer churn and building a predictive model
to help the telecom company retain customers.
Understand the relationships between factors and churn to better;

i.  Predict if a customer is likely to churn.


ii. Identify what factors contribute to churn so the company can take action to reduce it.

## 2. Data Collection:
-> *Data Source* : [Source](https://www.kaggle.com/royjafari/customer-churn) for data  and [source](https://archive.ics.uci.edu/ml/datasets/Iranian+Churn+Dataset) of dataset description.


-> The data consist of 14 columns and 3150 rows [see data_dict.md file]

### 2.1. Install Packages and Import Necessary Libraries



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')